[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/finance-data-analysis/blob/main/3.3%20%EB%84%A4%EC%9D%B4%EB%B2%84%EA%B8%88%EC%9C%B5%20%EA%B0%9C%EB%B3%84%EC%A2%85%EB%AA%A9%20%EC%88%98%EC%A7%91-input.ipynb)


## 네이버 금융 개별종목 수집
* FinanceDataReader를 통해 수집했던 데이터를 네이버 증권 웹 페이지를 통해 직접 수집합니다.


### Keyword

* html 파일 읽어오기
    * pd.read_html(url, encoding="cp949")

* 결측 데이터 제거하기(axis 0:행, 1:열)
    * table[0].dropna()

* 데이터 프레임 합치기
    * pd.concat([df1, df2, df3])

* 중복데이터 제거
    * df.drop_duplicates()

* 과학적 기수법
    * 1.210000e+02 => 121

* 날짜 column의 첫 row값 확인
    * date = df.iloc[0]["날짜"]

* 파일로 저장하기 
    * df.to_csv(file_name, index=False)

* 파일 읽어오기
    * pd.read_csv(file_name)

## 수집할 페이지 보기

* 네이버 금융 국내증시 : https://finance.naver.com/sise/
* 2020년 주요 상장종목
    * 빅히트 : https://finance.naver.com/item/main.nhn?code=352820
    * 카카오게임즈 : https://finance.naver.com/item/main.nhn?code=293490
    * SK바이오팜 : https://finance.naver.com/item/main.nhn?code=326030

## 라이브러리 로드

In [1]:
# 라이브러리 로드
import pandas as pd
import numpy as np

## 수집할 URL 정하기

In [2]:
# 종목번호와 상장사 이름을 item_code와 item_name으로 설정
# item_code = "352820"
# item_name = "하이브"

item_code = "326030"
item_name = "SK바이오팜"

# 종목 URL 만들기
# url = "https://finance.naver.com/item/sise.nhn?code=005930"
url = "https://finance.naver.com/item/sise_day.nhn?code=005930&page=10"

## requests를 통한 HTTP 요청
* [Requests: HTTP for Humans™ — Requests documentation](https://requests.readthedocs.io/en/master/)
* [Quickstart — Requests documentation # custom-headers](https://requests.readthedocs.io/en/latest/user/quickstart/#custom-headers)

In [3]:
import requests

In [4]:
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"}
print(type(headers))
headers

<class 'dict'>


{'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'}

In [5]:
reponse = requests.get(url, headers=headers)
# reponse.text

## BeautifulSoup 을 통한 table 태그 찾기

* [Beautiful Soup Documentation — Beautiful Soup 4.9.0 documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [6]:
from bs4 import BeautifulSoup as bs

html = bs(reponse.text, "lxml")

In [7]:
tables = html.select("table")
len(tables)

2

## pandas 코드 한 줄로 데이터 수집하기

In [8]:
print(url)

https://finance.naver.com/item/sise_day.nhn?code=005930&page=10


In [9]:
# read_html을 이용하여 url의 page내의 값을 DataFrame으로 받아옵니다.
# cp949는 한글 인코딩을 위해 사용합니다. 기본 인코딩 설정은 utf-8 이며, 
# 네이버의 일별 시세는 cp949 인코딩으로 불러올 수 있습니다.
# 데이터를 로드 했을 때 한글 인코딩이 깨진다면 대부분 cp949 로 불러올 수 있습니다.
# table

table = pd.read_html(str(tables))
table

[            날짜       종가     전일비       시가       고가       저가         거래량
 0          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 1   2021.03.22  82000.0   100.0  82000.0  82300.0  81700.0  12670506.0
 2   2021.03.19  81900.0  1000.0  82100.0  82500.0  81800.0  15869700.0
 3   2021.03.18  82900.0   600.0  82800.0  83800.0  82600.0  18585244.0
 4   2021.03.17  82300.0   500.0  82800.0  82900.0  82000.0  11625146.0
 5   2021.03.16  82800.0  1000.0  82200.0  83000.0  82100.0  12293537.0
 6          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 7          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 8          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 9   2021.03.15  81800.0  1000.0  82800.0  82900.0  81800.0  13735798.0
 10  2021.03.12  82800.0   800.0  83100.0  83500.0  82400.0  13530573.0
 11  2021.03.11  82000.0  1100.0  81000.0  82500.0  81000.0  23818297.0
 12  2021.03.10  80900.0   500.0  82400.0  82500.0  80700.0  200

In [10]:
# table[0]와 table[1]을 확인하여 보면 table[0]에 필요한 데이터들이 있습니다.
table[1]

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,맨앞,1,2,3,4,5,6,7,8,9,10,다음,맨뒤


In [11]:
# dropna를 통해 결측치가 들어있는 row를 제거합니다.
temp = table[0].dropna()
temp

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.03.22,82000.0,100.0,82000.0,82300.0,81700.0,12670506.0
2,2021.03.19,81900.0,1000.0,82100.0,82500.0,81800.0,15869700.0
3,2021.03.18,82900.0,600.0,82800.0,83800.0,82600.0,18585244.0
4,2021.03.17,82300.0,500.0,82800.0,82900.0,82000.0,11625146.0
5,2021.03.16,82800.0,1000.0,82200.0,83000.0,82100.0,12293537.0
9,2021.03.15,81800.0,1000.0,82800.0,82900.0,81800.0,13735798.0
10,2021.03.12,82800.0,800.0,83100.0,83500.0,82400.0,13530573.0
11,2021.03.11,82000.0,1100.0,81000.0,82500.0,81000.0,23818297.0
12,2021.03.10,80900.0,500.0,82400.0,82500.0,80700.0,20038422.0
13,2021.03.09,81400.0,600.0,81400.0,81900.0,80600.0,25420764.0


## 페이지별 데이터 수집 함수 만들기

In [12]:
print(url)

https://finance.naver.com/item/sise_day.nhn?code=005930&page=10


In [13]:
headers

{'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'}

In [103]:
# 종목 번호를 이용해 page에 따라 데이터를 읽어오는 함수
# """ 는 이 두개 사이의 행들은 주석 처리되며, 함수의 docstring 으로 사용됩니다.

def get_day_list(item_code, page_no):
    """
    일자별 시세를 페이지별로 수집
    """ 
    url = f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}"
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36'}
    response = requests.get(url, headers=headers)
    html = bs(response.text, "lxml")
    tables = html.select("table")
    table = pd.read_html(str(tables))
    df_page = table[0].dropna()
    return df_page

In [15]:
# 함수가 잘 만들어졌는지 확인하기
page_no = 300
item_code = "068270"
get_day_list(item_code, page_no)

https://finance.naver.com/item/sise_day.nhn?code=068270&page=300


,날짜,종가,전일비,시가,고가,저가,거래량
1,2009.06.24,16950.0,400.0,17450.0,17450.0,16200.0,1023682.0
2,2009.06.23,17350.0,500.0,17600.0,17700.0,17150.0,916680.0
3,2009.06.22,17850.0,100.0,17750.0,18100.0,17650.0,737032.0
4,2009.06.19,17750.0,200.0,17800.0,18250.0,17500.0,1191738.0
5,2009.06.18,17550.0,50.0,17900.0,18300.0,17450.0,1625660.0
9,2009.06.17,17500.0,500.0,17900.0,17950.0,17400.0,1033343.0
10,2009.06.16,18000.0,250.0,17900.0,18200.0,17850.0,747423.0
11,2009.06.15,18250.0,500.0,18800.0,18850.0,18200.0,678174.0
12,2009.06.12,18750.0,100.0,18900.0,19050.0,18550.0,815065.0
13,2009.06.11,18850.0,400.0,18300.0,19000.0,18200.0,1288496.0


In [25]:
def find_day_news(item_code, page_num):
    """
    일자별 종목 뉴스 수집
    """
    url = f"https://finance.naver.com/item/news_news.nhn?code={item_code}&page={page_num}&sm=title_entity_id.basic&clusterId="
    headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"}
    response = requests.get(url, headers = headers)
    html = bs(response.text, "lxml")
    tables = html.select("table")
    table = pd.read_html(str(tables))
    news_page = table[0]
    news_page_result = news_page[~news_page.get("제목").str.contains("연관기사")]
    return news_page_result


In [ ]:
find_day_news(item_code, page_num=1)

In [27]:
### 종목 토론 수집
def get_day_debate(item_code, page_num):
    """
    종목 토론 수집
    """
    url = f"https://finance.naver.com/item/board.nhn?code=005930&page={page_num}"
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"}
    response = requests.get(url, headers = headers)
    html = bs(response.text, "lxml")
    tables = html.select("table")
    table = pd.read_html(str(tables))
    table_page = table[1]
    df_debate = table_page.iloc[:,:6]
    debate_page_result = df_debate.dropna()
    return debate_page_result

In [ ]:
get_day_debate(item_code, page_num=1)

## 반복문을 통한 전체 일자 데이터 수집하기
* (주의) 기간이 긴 데이터를 수집할때는 서버에 부담을 주지 않기 위해 time.sleep()값을 주세요.

In [34]:
print(1)
time.sleep(0.1)
print(2)

1
2


In [35]:
item_code

'068270'

In [36]:
url

'https://finance.naver.com/item/sise_day.nhn?code=005930&page=10'

In [43]:
import time
from tqdm import trange
# web page 시작번호
start_no = 1
end_no = 10

# 데이터를 저장할 빈 변수 선언
item_list = []
for page_no in trange(start_no, end_no+1):
    temp = get_day_list(item_code, page_no)
    item_list.append(temp)

# item_list

100%|██████████| 10/10 [00:01<00:00,  8.42it/s]


In [66]:
page_no = 28
get_day_list(item_code, page_no).shape[0]

7

In [109]:
temp = get_day_list(item_code, page_no)
temp.tail(2)

,날짜,종가,전일비,시가,고가,저가,거래량
9,2020.10.16,200500.0,57500.0,253000.0,253000.0,199000.0,4504395.0
10,2020.10.15,258000.0,12000.0,270000.0,351000.0,253500.0,6557212.0


'2020.10.15'

In [116]:
# item_code = "326030"
# item_name = "SK바이오팜"

item_code = "352820"
item_name = "하이브"

# 페이지별 결과를 받을 리스트를 초기화
item_list = []

# while
page_no = 1

while True:
    # 수집함수
    temp = get_day_list(item_code, page_no)
    
    # 두번쨰 페이지부터 확인
    if page_no > 1 :
        pre_date = item_list[-1]['날짜'].tolist()[0]
        now_date = temp['날짜'].tolist()[0]
        if pre_date == now_date :
            break
            
    item_list.append(temp)
    
    # 서버에 부담을 주지 않기 위해 시간차를 두고 가져옵니다.
    random_time = np.random.uniform(0.1, 0.5)
    time.sleep(random_time)
    
    # 페이지 번호를 하나씩 증가 시키고
    page_no = page_no + 1
    
    # 5페이지 단위로 현재 몇 페이지인지를 출력
    if page_no % 5 == 0:
        print(page_no)
    
    # 일별시세는 10개씩 행이 있기 때문에 행이 10개보다 적으면 마지막 페이지로 보고 break 
#     if temp.shape[0] != 10 :
#         break

5
10
15
20


In [117]:
item_list

[            날짜        종가     전일비        시가        고가        저가       거래량
 1   2021.07.28  302500.0  4000.0  299000.0  303500.0  297500.0   61555.0
 2   2021.07.27  298500.0   500.0  301000.0  302000.0  296500.0  129698.0
 3   2021.07.26  299000.0  4000.0  305000.0  307000.0  298500.0  170632.0
 4   2021.07.23  303000.0  1500.0  300500.0  306500.0  299000.0  196806.0
 5   2021.07.22  301500.0  3000.0  301500.0  306500.0  298500.0  254160.0
 9   2021.07.21  298500.0  7500.0  311000.0  312000.0  296500.0  388660.0
 10  2021.07.20  306000.0  9000.0  311500.0  318500.0  299000.0  484428.0
 11  2021.07.19  315000.0  2500.0  317000.0  317500.0  310500.0  209873.0
 12  2021.07.16  317500.0  2000.0  319000.0  319000.0  314500.0  227008.0
 13  2021.07.15  319500.0   500.0  320000.0  322000.0  315500.0  238471.0,
             날짜        종가      전일비        시가        고가        저가       거래량
 1   2021.07.14  319000.0  10000.0  312000.0  323000.0  309000.0  530995.0
 2   2021.07.13  309000.0   2000.0 

## 수집한 데이터 하나의 데이터프레임으로 합치기

<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

In [118]:
# DataFrame들이 list의 형태로 저장된 것을 concat을 이용하여 합치면서 하나의 DataFrame으로 만듭니다.
df = pd.concat(item_list)

<img src="https://pandas.pydata.org/docs/_images/08_concat_row.svg">

In [119]:
# head와 tail로 데이터의 일부를 가져와서 봅니다.
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.07.28,302500.0,4000.0,299000.0,303500.0,297500.0,61555.0
2,2021.07.27,298500.0,500.0,301000.0,302000.0,296500.0,129698.0
3,2021.07.26,299000.0,4000.0,305000.0,307000.0,298500.0,170632.0
4,2021.07.23,303000.0,1500.0,300500.0,306500.0,299000.0,196806.0
5,2021.07.22,301500.0,3000.0,301500.0,306500.0,298500.0,254160.0


In [120]:
df.tail()

,날짜,종가,전일비,시가,고가,저가,거래량
3,2020.10.21,179000.0,3500.0,180500.0,185500.0,177000.0,1658643.0
4,2020.10.20,182500.0,6500.0,190000.0,193500.0,176000.0,1811616.0
5,2020.10.19,189000.0,11500.0,198500.0,202500.0,187000.0,1977996.0
9,2020.10.16,200500.0,57500.0,253000.0,253000.0,199000.0,4504395.0
10,2020.10.15,258000.0,12000.0,270000.0,351000.0,253500.0,6557212.0


## 데이터프레임에 종목코드와 종목명을 추가하기
* 파생변수 만들기

In [121]:
# '종목코드'와 '종목명' column을 추가하면서 각각 item_code와 item_name 값을 입력합니다.
df["종목코드"] = item_code
df["종목명"] = item_name

In [122]:
df.head(2)

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드,종목명
1,2021.07.28,302500.0,4000.0,299000.0,303500.0,297500.0,61555.0,352820,하이브
2,2021.07.27,298500.0,500.0,301000.0,302000.0,296500.0,129698.0,352820,하이브


## 컬럼 순서 변경하기

In [142]:
# DataFrame에서 column 들의 이름을 순서를 조정하여 column순서를 변경할 수 있습니다.
df.columns

Index(['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량'], dtype='object')

In [143]:
cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
df = df[cols]
df.head(2)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,352820,하이브,2021.07.28,302500.0,4000.0,299000.0,303500.0,297500.0,61555.0
2,352820,하이브,2021.07.27,298500.0,500.0,301000.0,302000.0,296500.0,129698.0


## 중복데이터 제거하기
* drop_duplicates 를 통해 중복된 데이터가 있다면 제거합니다.

In [126]:
# drop_duplicates : row들 끼리 data를 비교하여 같은 값이 있으면 row중 하나를 삭제
# df.shape를 전/후로 출력하여 삭제된 row가 있는지 확인
print(df.shape)
df = df.drop_duplicates()
df.shape

(197, 9)


(197, 9)

## 기술통계값 구하기

In [127]:
# describe는 기본적으로 수치데이터에 대한 기술통계값을 구하게 됩니다.
df.describe()

,종가,전일비,시가,고가,저가,거래량
count,197.000000,197.000000,197.000000,197.000000,197.000000,1.970000e+02
mean,226573.604061,5954.314721,227015.228426,233012.690355,221573.604061,5.570544e+05
std,50458.797232,6206.517654,50565.650109,51781.118923,49821.425128,7.281001e+05
min,142000.000000,0.000000,142500.000000,146500.000000,141000.000000,6.155500e+04
25%,180000.000000,1500.000000,180000.000000,183000.000000,175500.000000,2.154120e+05
50%,232500.000000,4000.000000,232000.000000,241000.000000,226500.000000,3.555120e+05
75%,260500.000000,8000.000000,262000.000000,266000.000000,255500.000000,5.811950e+05
max,324500.000000,57500.000000,334500.000000,351000.000000,318500.000000,6.557212e+06


* 참고 : E notation
[과학적 기수법 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EA%B3%BC%ED%95%99%EC%A0%81_%EA%B8%B0%EC%88%98%EB%B2%95)

과학적 기수법, 과학적 표기법(scientific notation, scientific form, standard index form, standard form)은 너무 크거나 너무 작은 숫자들을 십진법으로 편하게 작성하여 표현하는 방법이다. 과학자, 수학자, 공학자들이 공통적으로 사용하는데, 부분적인 이유는 특정한 산술을 단순화시켜 주기 때문이다. 과학 계산기에서는 "SCI" 디스플레이 모드라는 이름으로 알려져 있다.

In [128]:
# 거래량의 과학적 기수법 읽기
6.155500e+04

61555.0

## 최근 날짜 구해서 파일명 만들기

In [141]:
df.head(1)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,352820,하이브,2021.07.28,302500.0,4000.0,299000.0,303500.0,297500.0,61555.0


In [135]:
# 날짜 column의 첫 row값 확인
date = df.iloc[0, 2]
date

'2021.07.28'

In [139]:
# 종목명, 종목코드, 날짜를 이름으로 하는 csv 파일명 만들기
file_name = f"{item_name}_{item_code}_{date}.csv"
file_name

'하이브_352820_2021.07.28.csv'

## 파일로 저장하기

In [146]:
# 파일로 저장하기 
# index=False 로 데이터프레임의 기본 index 는 저장하지 않도록 합니다.
df.to_csv(file_name, index=False)

In [147]:
# 제대로 저장되었는지 파일을 읽어서 확인합니다.
pd.read_csv(file_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,352820,하이브,2021.07.28,302500.0,4000.0,299000.0,303500.0,297500.0,61555.0
1,352820,하이브,2021.07.27,298500.0,500.0,301000.0,302000.0,296500.0,129698.0
2,352820,하이브,2021.07.26,299000.0,4000.0,305000.0,307000.0,298500.0,170632.0
3,352820,하이브,2021.07.23,303000.0,1500.0,300500.0,306500.0,299000.0,196806.0
4,352820,하이브,2021.07.22,301500.0,3000.0,301500.0,306500.0,298500.0,254160.0
...,...,...,...,...,...,...,...,...,...
192,352820,하이브,2020.10.21,179000.0,3500.0,180500.0,185500.0,177000.0,1658643.0
193,352820,하이브,2020.10.20,182500.0,6500.0,190000.0,193500.0,176000.0,1811616.0
194,352820,하이브,2020.10.19,189000.0,11500.0,198500.0,202500.0,187000.0,1977996.0
195,352820,하이브,2020.10.16,200500.0,57500.0,253000.0,253000.0,199000.0,4504395.0


## 전체 과정을 하나의 함수로 만들기

In [149]:
def get_item_list(item_code, item_name):
    """
    일별 시세를 수집하는 함수
    """
    
    # 페이지별 결과를 받을 리스트를 초기화
    item_list = []

    # 페이지 번호 초기화
    page_no = 1

    while True:
        # 수집함수
        temp = get_day_list(item_code, page_no)

        # 두번쨰 페이지부터 확인
        if page_no > 1 :
            pre_date = item_list[-1]['날짜'].tolist()[0]
            now_date = temp['날짜'].tolist()[0]
            if pre_date == now_date :
                break

        item_list.append(temp)

        # 서버에 부담을 주지 않기 위해 시간차를 두고 가져옵니다.
        random_time = np.random.uniform(0.1, 0.5)
        time.sleep(random_time)

        # 페이지 번호를 하나씩 증가 시키고
        page_no = page_no + 1

        # 5페이지 단위로 현재 몇 페이지인지를 출력
        if page_no % 5 == 0:
            print(page_no)

    # 전처리
    df = pd.concat(item_list)
    df["종목코드"] = item_code
    df["종목명"] = item_name
    cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
    df = df[cols].copy()
    
    # 파일저장
    date = df.iloc[0, 2]
    file_name = f"{item_name}_{item_code}_{date}.csv"
    df.to_csv(file_name, index=False)
    return True

In [150]:
# item_code = "352820"
# item_name = "하이브"
# 빅히트의 사명이 하이브로 변경되었습니다.

# item_code = "326030"
# item_name = "SK바이오팜"
item_code = "293490"
item_name = "카카오게임즈"
get_item_list(item_code, item_name)

5
10
15
20


True

## 내가 만든 함수의 기능과 소스코드가 궁금하다면?

In [151]:
# 도움말 보기
# get_item_list?

In [152]:
# 소스코드 보기
# get_item_list??
